# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="C:\Banana_Windows_x86_64\Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
import torch
from dqn_agent import Agent
agent = Agent(37,4)
results=[]    #results to store
from collections import deque
rollingresults=deque(maxlen=100) #100 last results to calculate mean

In [7]:
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
eps = 1.                                           # epsilon
episode = 1                                        # episode no
while episode<2000:
    action = agent.act(eps=eps,state=state)        # select an action
    env_info = env.step(int(action))[brain_name]   # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    agent.step(state,action,reward,next_state,done) # store state, and maybe learn something
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:        #if episode complete
        eps *= 0.995                  #decrease epsilon...
        if eps < .05:                 #...but not too small
            eps = .05
        results.append([episode,score])              #save and print results
        rollingresults.append(score)
        print("Episode {}, score {} (mean {}), epsilon {}                      ".format(episode,score,np.mean(rollingresults),eps),end="\r")
        if np.mean(rollingresults) > 13:    #if epsiode solved - exit from cycle
            break
        if episode % 50 == 0:   #sometimes we just save model and print results from new row
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.ep{}.pth'.format(episode))
            print()
        episode += 1   #starting new episode
        score=0
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.trained.pth')
print("\nSolved in {} episodes".format(episode))

Episode 50, score 1.0 (mean 0.22), epsilon 0.778312557068642                                       
Episode 100, score 5.0 (mean 0.99), epsilon 0.6057704364907278                                    
Episode 150, score 5.0 (mean 3.13), epsilon 0.47147873742168567                      
Episode 200, score 7.0 (mean 5.24), epsilon 0.3669578217261671                        
Episode 250, score 10.0 (mean 6.8), epsilon 0.285607880564032                         
Episode 300, score 9.0 (mean 7.81), epsilon 0.22229219984074702                       
Episode 350, score 10.0 (mean 9.02), epsilon 0.1730128104744653                       
Episode 400, score 4.0 (mean 10.52), epsilon 0.1346580429260134                        
Episode 450, score 16.0 (mean 11.63), epsilon 0.10480604571960442                      
Episode 500, score 15.0 (mean 12.62), epsilon 0.08157186144027828                      
Episode 537, score 20.0 (mean 13.08), epsilon 0.0677632708131484                       
Solved in 537 e

Saving results for further usage

In [8]:
import pickle as pkl
with open('results.pkl','wb') as a:
    pkl.dump(file=a,obj=results)

Drawing learning curve

In [9]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
pd.DataFrame([x[1] for x in results]).plot()
plt.show()

<Figure size 640x480 with 1 Axes>

Replay

In [10]:
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.trained.pth'))
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = agent.act(eps=0,state=state)        # select an action
    env_info = env.step(int(action))[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:
        print("Score {}".format(score))
        break 


Score 14.0


In [11]:
env.close()